# AM-SS Data Demodulation 

In [377]:
import h5py
import json
import math
import numpy as np
from scipy import signal

import matplotlib.pyplot as plt
from matplotlib import rc
import datetime
import scipy.signal as signal
#import pandas as pd
#import seaborn as sns
import statistics as stats
import os
import scipy.io
import h5py
# from scipy.spatial.distance import hamming
# from time import sleep

rc('xtick', labelsize=14) 
rc('ytick', labelsize=14)

In [378]:
'''
Protocol parameters!!!
'''
packet = 6000
samples = 600
pseudonym_len = 28
TX_length = 11200
repeat = 10
message = 'STOP'

In [379]:
def get_time_string(timestamp):
    '''
    Helper function to get data and time from timestamp
    INPUT: timestamp
    OUTPUT: data and time. Example: 01-04-2023, 19:50:27
    '''
    date_time = datetime.datetime.fromtimestamp(int(timestamp))
    return date_time.strftime("%m-%d-%Y, %H:%M:%S")

In [380]:
def JsonLoad(folder, json_file):
    '''
    Load parameters from the saved json file
    INPUT
    ----
        folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
        json_file: the json file with all the specifications. Example: '/save_iq_w_tx_gold.json'
    OUTPUT
    ----
        samps_per_chip: samples per chip
        wotxrepeat: number of repeating IQ sample collection w/o transmission. Used as an input to 
        traverse_dataset() func
        rxrate: sampling rate at the receiver side
    '''
    #config_file = folder+'/'+json_file
    #config_file = ""+"/"+str(folder)+"/save_iq_w_tx_file.json"
    config_dict = json.load(open(json_file))[0]
    nsamps = config_dict['nsamps']
    rxrate = config_dict['rxrate']
    rxfreq = config_dict['rxfreq']
    wotxrepeat = config_dict['wotxrepeat']
    rxrepeat = config_dict['rxrepeat']
    txnodes = config_dict['txclients']
    rxnodes = config_dict['rxclients']

    return rxrepeat, rxrate, txnodes, rxnodes

In [381]:
def traverse_dataset(meas_folder):
    '''
    Load data from hdf5 format measurement file
    INPUT
    ----
        meas_folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
    OUTPUT
    ----
        data: Collected IQ samples w/ transmission. It is indexed by the transmitter name
        noise: Collected IQ samples w/o transmission. It is indexed by the transmitter name
        txrxloc: transmitter and receiver names
    '''
    data = {}
    noise = {}
    txrxloc = {}

    dataset = h5py.File(meas_folder + '/measurements.hdf5', "r") #meas_folder
    #print("Dataset meta data:", list(dataset.attrs.items()))
    for cmd in dataset.keys():
        #print("Command:", cmd)
        if cmd == 'saveiq':
            cmd_time = list(dataset[cmd].keys())[0]
           # print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for rx_gain in dataset[cmd][cmd_time].keys():
               # print("   RX gain:", rx_gain)
                for rx in dataset[cmd][cmd_time][rx_gain].keys():
                    print("")
                    #print("     RX:", rx)
                    #print("       Measurement items:", list(dataset[cmd][cmd_time][rx_gain][rx].keys()))
        elif cmd == 'saveiq_w_tx':
            cmd_time = list(dataset[cmd].keys())[0]
            #print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for tx in dataset[cmd][cmd_time].keys():
                #print("   TX:", tx)
                
                if tx == 'wo_tx':
                    for rx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("       RX gain:", rx_gain)
                       # print(dataset[cmd][cmd_time][tx][rx_gain].keys())
                        for rx in dataset[cmd][cmd_time][tx][rx_gain].keys():
                            #print("         RX:", rx)
                            #print("           Measurement items:", list(dataset[cmd][cmd_time][tx][rx_gain][rx].keys()))
                            repeat = np.shape(dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'])[0]
                            #print("         repeat", repeat)

                            samplesNotx =  dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'][:repeat, :]
                            namelist = rx.split('-')
                            noise[namelist[1]] = samplesNotx
                else:
                    for tx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("     TX gain:", tx_gain)
                        for rx_gain in dataset[cmd][cmd_time][tx][tx_gain].keys():
                            #print("       RX gain:", rx_gain)
                            #print(dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys())
                            for rx in dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys():
                                repeat = np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'])[0]
                                #print("         RX:", rx, "; samples shape", np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples']))
                                #print("         Measurement items:", list(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx].keys()))
                                # print("         rxloc", (dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxloc'][0]))
                                # peak avg check
                                
                                txrxloc.setdefault(tx, []).extend([rx]*repeat)
                                rxsamples = dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'][:repeat, :]
                                data.setdefault(tx, []).append(np.array(rxsamples))

        else:                       
            print('Unsupported command: ', cmd)

    return data, noise, txrxloc

In [382]:
# PURPOSE: perform preamble synchronization
#          Uses the (complex-valued) preamble signal. The cross-correlation 
#          of the preamble signal and the received signal (at the time
#          when the preamble is received) should have highest magnitude
#          at the index delay where the preamble approximately starts.  
# INPUT:   rx0: received signal (with a frequency offset)
#          preambleSignal: complex, known, transmitted preamble signal 
# OUTPUT:  lagIndex: the index of rx0 where the preamble signal has highest 
#              cross-correlation
#
def crossCorrelationMax(rx0, preambleSignal):

    # Cross correlate with the preamble to find it in the noisy signal
    lags      = signal.correlation_lags(len(rx0), len(preambleSignal), mode='valid')
    xcorr_out = signal.correlate(rx0, preambleSignal, mode='valid')
    xcorr_mag = np.abs(xcorr_out)
    # Don't let it sync to the end of the packet.
    packetLenSamples = 168800
    maxIndex = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex = lags[maxIndex]

    #print('Max crosscorrelation with preamble at lag ' + str(lagIndex))

    # # Plot the selected signal.
    # plt.figure()
    # fig, subfigs = plt.subplots(2,1)
    # subfigs[0].plot(np.real(rx0), label='Real RX Signal')
    # subfigs[0].plot(np.imag(rx0), label='Imag RX Signal')
    # scale_factor = np.mean(np.abs(rx0))/np.mean(np.abs(preambleSignal))
    # subfigs[0].plot(range(lagIndex, lagIndex + len(preambleSignal)), scale_factor*np.real(preambleSignal), label='Preamble')
    # subfigs[0].legend()
    # subfigs[1].plot(lags, xcorr_mag, label='|X-Correlation|')
    # plt.xlabel('Sample Index', fontsize=14)
    # plt.tight_layout()
    # plt.show()

    return lagIndex

In [383]:
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [384]:
def binvector2str(binvector):
    #binvector = binvector[0]
    length = len(binvector)
    eps = np.finfo('float').eps
    if abs(length/7 - round(length/7)) > eps:
        print('Length of bit stream must be a multiple of 7 to convert to a string.')
    # Each character requires 7 bits in standard ASCII
    num_characters = round(length/7)
    # Maximum value is first in the vector. Otherwise would use 0:1:length-1
    start = 6
    bin_values = []
    while start >= 0:
        bin_values.append(int(math.pow(2,start)))
        start = start - 1
    bin_values = np.array(bin_values)
    bin_values = np.transpose(bin_values)
    str_out = '' # Initialize character vector
    for i in range(num_characters):
        single_char = binvector[i*7:i*7+7]
        value = 0
        for counter in range(len(single_char)):
            value = value + (int(single_char[counter]) * int(bin_values[counter]))
        str_out += chr(int(value))
    return str_out

In [386]:
'''
calculates the distance between the recoded pseudonym bits and the transmitted(true) pseudonym bits.
'''
def Distance(X,Y):
    if len(X) != len(Y):
        print('Warning: arrays have different dimensions')
    count = 0
    for i in range(len(X)):
        if X[i]!= Y[i]:
            count +=1
    return count

In [387]:
'''Use preamble to estimate start of OFDM packet'''
def Generate_HTSTF():
    data = scipy.io.loadmat('HTSTF.mat')
    new_data = data['stf'].reshape((1,len(data['stf'])))
    preamble = np.tile(new_data,10)[0]
    return preamble

In [388]:
preambleSignal = Generate_HTSTF()

In [390]:
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [391]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Calculate_Eb_No_Passive(x):
    
    # Load parameters from the JSON file which describe what was measured
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    # samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-browning-comp'

    #Calculate SNR
    Noise = rx_noise['browning'][0]  # measure the RX power while the TX is turned off
    noise_power = np.mean(abs(Noise)**2)
    
    P_s = 0 #received power

    for i in range(repeat):    
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][i]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]
       
        P_s += np.mean(abs(Rx_signal)**2) 
    return P_s/repeat, noise_power

In [392]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''

def Calculate_Pseudonym_BER(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
#     samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-browning-comp'
    # rxloc = 'cbrssdr1-browning-comp'
    # initialize error
    pseudonym_BER = 0
    P_s = 0
    count = 0
    for i in range(repeat):    
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][i]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]

        estimate_pseudonym = Matched_Filter_Pseudonym_Detection_Algorithm(Rx_signal)
        print(binvector2str(estimate_pseudonym))

        pseudonym_BER += Distance(text2bits('STOP'),estimate_pseudonym)
    return pseudonym_BER

In [393]:
# Load collected IQ samples from file
def readCom(file_path):
    return np.fromfile(file_path, dtype=np.complex64)

In [394]:
'''
Computes the average pseudonym error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Probability_Pseudonym_Detection(x):
    folders = Extract_Folders(x)[0]
    num_folder = len(folders)
    pseudonym_BER = 0
    signal = 0
    noise = 0
    noise_power = []
    signal_power = []
    for i in range(len(folders)):
        s,n = Calculate_Eb_No_Passive(x + '/'+folders[i])
        noise_power.append(n)
        signal_power.append(s)
        signal += s
        noise += n
    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))

    plt.plot(noise_power)
    plt.show()
    plt.plot(signal_power)
    plt.show()
    print('The SNR at the Passive RX is:',round(Eb_No,2), 'dB \n')

    for i in range(len(folders)):

        BER = Calculate_Pseudonym_BER(x + '/'+folders[i])
        pseudonym_BER += BER
    
    p_bit_error = pseudonym_BER/(num_folder*repeat*pseudonym_len)
    return p_bit_error, Eb_No

# OFDM Data Demodulation at Intended Receiver Below

In [396]:
'''
Protocol parameters!!!
'''
FFT = 64 # FFT size for extracting IQ sample in each subcarrier
OFDM_size = 80 # OFDM symbol with cyclic prefix
data_size = 48 # data_subcarriers
mess_length = 560
actual_message='I have worked in SPAN Lab for the last three & half years.I implemented Pseudonymetry in POWDER.'
num_bits = 672

In [397]:
CP = 16  # 25% cyclic prefix
pilotValue = 3+3j # known values for pilots

In [398]:
allCarriers = np.arange(FFT)  # indices of all subcarriers ([0, 1, ... K-1])
pilotCarriers = np.array([7,21,36,50]) # Pilots indices.
guardCarriers = np.array([0,1,2,3,4,5,58,59,60,61,62,63]) # 6 subcarrier guard bands on each side
nondataCarriers = np.concatenate([pilotCarriers,guardCarriers])
dataCarriers = np.delete(allCarriers, nondataCarriers) # data carriers are allCarriers -(pilot+guard carriers)

In [399]:
# Purpose: Convert M-ary data to binary data
#          each m-ary value input in "data" is converted to
#          log2(M) binary values.
# INPUT: M-ary digit vector
# OUTPUT: Binary digit vector, with length equal to the number
#         of values in data multiplied by log2(M)
def mary2binary(data, M):
    length = len(data) # number of values in data
    log2M = round(np.log2(M)) # integer number of bits per data value
    format_string = '0' + str(log2M) + 'b'
    binarydata = np.zeros((1,length*log2M))
    count = 0
    for each in data:
        binval = format(int(each), format_string)
        for i in range(log2M):
            binarydata[0][count+i] = int(binval[i])
        count = count + log2M
    return binarydata

In [400]:
# PURPOSE: Find the symbols which are closest in the complex plane 
#          to the measured complex received signal values.
# INPUT:   Received r_hat values (output of matched filter downsampled),
#          and possible signal space complex values. 
# OUTPUT:  m-ary symbol indices in 0...length(outputVec)-1
def findClosestComplex(r_hat, outputVec):
    # outputVec is a 4-length vector for QPSK, would be M for M-QAM or M-PSK.
    # This checks, one symbol sample at a time,  which complex symbol value
    # is closest in the complex plane.
    data_out = [np.argmin(np.abs(r-outputVec)) for r in r_hat]
    return data_out

In [404]:
def Channel_Estimation(x):
    pilots = x[pilotCarriers] 
    H_at_pilots = pilots / pilotValue 
    
    # Perform interpolation between the pilot carriers to get an estimate of the channel in the data carriers. 
    H_abs=scipy.interpolate.interp1d(pilotCarriers, abs(H_at_pilots), kind='nearest', fill_value='extrapolate')(allCarriers)
    H_phase = scipy.interpolate.interp1d(pilotCarriers, np.angle(H_at_pilots), kind='nearest',fill_value = 'extrapolate')(allCarriers)
    H_estimate = H_abs * np.exp(1j*H_phase)
    
    return H_estimate

In [405]:
def Equalization(OFDM_demod, Hest):
    return OFDM_demod / Hest

In [406]:
'''PURPOSE: To recover the data signal'''
def OFDM_RX(data):
    for i in range(len(data)//(OFDM_size)):
        data_cp = data[i*(OFDM_size):(i+1)*(OFDM_size)]
        data_without_cp = data_cp[CP:]
        
        # Generate frequency domain signal
        OFDM_freq = np.fft.fft(data_without_cp,n=FFT)
        
        H_est = Channel_Estimation(OFDM_freq) # estimate the channel
        
        OFDM_est = Equalization(OFDM_freq,H_est) # sub-carrier equalization
        
        OFDM_data = OFDM_est[dataCarriers] # extract the data signal
        if i == 0:
            OFDM_swap =  OFDM_data
        else:
            OFDM_signal = np.concatenate((OFDM_swap,  OFDM_data))
            OFDM_swap = OFDM_signal
    return OFDM_signal

In [407]:
def QPSK_Data_Demodulation(x):
    outputVec = np.array([1+1j, -1+1j, 1-1j, -1-1j])
    data_error = 0.0
    for i in range(len(x)//mess_length):
        time_signal = x[i*mess_length:(i+1)*mess_length]
        detected_signal = OFDM_RX(time_signal)
       
        mary_out  = findClosestComplex(detected_signal, outputVec)
        data_bits  = mary2binary(mary_out, 4)[0]
        data_error += Distance(data_bits,text2bits(actual_message))
        # print(binvector2str(data_bits))
    return data_error

In [408]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Calculate_Eb_No_Intended(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'# select the TX
    rxloc = 'cbrssdr1-meb-comp' # select the RX
    # rxloc = 'cbrssdr1-browning-comp'
    
    #Calculate SNR
    Noise = rx_noise['meb'][0]
    # Noise = rx_noise['browning'][0]  # measure the RX power while the TX is turned off
    noise_power = np.mean(abs(Noise)**2)
    
    # initialize error
    
    P_s = 0

    for i in range(repeat):    
        repNum = i
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][repNum]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]

       
        P_s += np.mean(abs(Rx_signal)**2)

    signal_power = P_s/repeat   
    return signal_power, noise_power

In [409]:
'''
Checks if data bits are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''

def Calculate_Data_BER(x):
    # Load parameters from the JSON file which describe what was measured

    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
#     samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-meb-comp'
    # rxloc = 'cbrssdr1-browning-comp'

    # initialize error
    data_BER = 0.0
    
    for i in range(repeat):    
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][i]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:TX_length+start_of_data]

        data_BER += QPSK_Data_Demodulation(Rx_signal) 
        
    return data_BER

In [410]:
'''
Computes the average data bit error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Probability_Data_Demodulation(x):
    folders = Extract_Folders(x)[0]
    num_folder = len(folders)
    Data_BER = 0
    signal = 0
    noise = 0
    noise_power = []
    signal_power = []
    for i in range(len(folders)):
        print(folders[i])
        s,n = Calculate_Eb_No_Intended(x + '/'+folders[i])
        noise_power.append(n)
        signal_power.append(s)
        signal += s
        noise += n
    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))
    
    print('The SNR at the intended RX is:',round(Eb_No,2), 'dB \n')

    for i in range(len(folders)):
        Data_BER += Calculate_Data_BER(x + '/'+folders[i])
    
    d_bit_error = Data_BER/(num_folder*repeat*num_bits*300)
    return d_bit_error, Eb_No 

# Results Below

In [413]:
Probability_Data_Demodulation("data")